In [92]:
import boto3
import json
import base64
import time
import random
import sys

DATA_SIZE = 320000
ITERATIONS = 1000

response_times = []

# open boto3 lambda client (requires ~/.aws/config and ~/.aws/credentials to be setup)
client = boto3.client('lambda')

for iteration in range(ITERATIONS):
    if iteration > 0 and iteration % 100 == 0:
        print(iteration, sum(response_times)/len(response_times))
    random_floats = [random.uniform(-10, 80) for _ in range(DATA_SIZE)]
    request_payload = {
        "function": "sum",
        "data": random_floats
    }
    request_payload_bytes = bytes(json.dumps(request_payload), encoding='utf-8')

    start_time = time.time()
    response = client.invoke(
        FunctionName='Cider_Compute',
        InvocationType='RequestResponse', # synchronous invocation (wait for a response)
        Payload=request_payload_bytes,
        Qualifier='1' # Cider_Compute published version 1
    ) 

    if response['StatusCode'] == 200:
        response_bytes = response['Payload'].read()
        response_times.append(time.time() - start_time)
    else:
        print("ERROR:", response['StatusCode'], response['FunctionError'])

100 2.2892933082580567
200 2.2590131843090058
300 2.245024733543396
400 2.21520581305027
500 2.2010088095664977
600 2.2078145611286164
700 2.224548670904977
800 2.221131690442562
900 2.225392626656426


In [95]:
import statistics

statistics.median(response_times)

2.092103600502014